In [1]:
from datetime import datetime
from aeon.classification.convolution_based import RocketClassifier
from aeon.classification.feature_based import Catch22Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from aeon.classification.deep_learning import InceptionTimeClassifier
from aeon.classification.deep_learning import ResNetClassifier

from f1_etl import (
    DataConfig,
    SessionConfig,
    create_safety_car_dataset,
)
from f1_etl.train import (
    ModelEvaluationSuite,
    create_metadata_from_f1_dataset,
    prepare_data_with_validation,
    create_model_metadata,
    train_and_validate_model,
    evaluate_on_external_dataset,
    compare_performance_across_datasets,
)

In [12]:
drivers=["1"]
window_size=100
prediction_horizon=10

# 1. Load dataset
data_config = DataConfig(
    sessions=[
        SessionConfig(2024, "Qatar Grand Prix", "R"),
        SessionConfig(2024, "Chinese Grand Prix", "R"),
        SessionConfig(2024, "Mexico City Grand Prix", "R"),
        SessionConfig(2024, "São Paulo Grand Prix", "R"),
        SessionConfig(2024, "Miami Grand Prix", "R"),
        SessionConfig(2024, "United States Grand Prix", "R"),
        SessionConfig(2024, "Monaco Grand Prix", "R"),
    ],
    drivers=drivers,
    include_weather=False,
)

# Using PCA with automatic component selection (95% variance)
dataset = create_safety_car_dataset(
    config=data_config,
    window_size=window_size,
    prediction_horizon=prediction_horizon,
    normalize=True,
    target_column="TrackStatus",
    feature_transform="pca",  # Enable PCA
    pca_variance_threshold=0.95,  # Keep 95% variance
)

2025-07-13 12:17:31,066 - f1_etl - INFO - Preprocessing configuration:
2025-07-13 12:17:31,069 - f1_etl - INFO -   Missing values: enabled (forward_fill)
2025-07-13 12:17:31,069 - f1_etl - INFO -   Normalization: enabled (standard)
2025-07-13 12:17:31,070 - f1_etl - INFO -   Feature transform: pca
2025-07-13 12:17:31,071 - f1_etl - INFO -   Resampling: disabled
2025-07-13 12:17:31,071 - f1_etl - INFO - Driver configuration:
2025-07-13 12:17:31,072 - f1_etl - INFO -   Global drivers: ['1']
2025-07-13 12:17:31,072 - f1_etl - INFO -   Qatar Grand Prix: ['1']
2025-07-13 12:17:31,073 - f1_etl - INFO -   Chinese Grand Prix: ['1']
2025-07-13 12:17:31,074 - f1_etl - INFO -   Mexico City Grand Prix: ['1']
2025-07-13 12:17:31,074 - f1_etl - INFO -   São Paulo Grand Prix: ['1']
2025-07-13 12:17:31,075 - f1_etl - INFO -   Miami Grand Prix: ['1']
2025-07-13 12:17:31,077 - f1_etl - INFO -   United States Grand Prix: ['1']
2025-07-13 12:17:31,077 - f1_etl - INFO -   Monaco Grand Prix: ['1']
events   

Loading session: 2024 Qatar Grand Prix R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '81', '63', '10', '55', '14', '24', '20', '4', '77', '44', '22', '30', '23', '27', '11', '18', '43', '31']
core           INFO 	Loading data for Chinese Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Loading session: 2024 Chinese Grand Prix R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:08.313000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '11', '16', '55', '63', '14', '81', '44', '27', '31', '23', '10', '24', '18', '20', '2', '3', '22', '77']
core           INFO 	Loading data for Mexico City Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app

...

req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '1', '4', '81', '63', '11', '27', '30', '43', '20', '10', '14', '22', '18', '23', '77', '31', '24', '44']
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Loading session: 2024 Monaco Grand Prix R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']
2025-07-13 12:17:47,815 - f1_etl - INFO - Creating new fixed vocabulary encoder
2025-07-13 12:17:47,959 - f1_etl - INFO - Applying PCA transformation to telemetry features
2025-07-13 12:17:47,960 - f1_etl - INFO - Applying PCA to 9 numeric features: ['Speed', 'RPM', 'nGear', 'Throttle', 'Brake', 'X', 'Y', 'Distance', 'DifferentialDistance']



📊 Track Status Analysis (training_data):
   green       : 452387 samples ( 84.0%)
   red         : 25292 samples (  4.7%)
   safety_car  : 36534 samples (  6.8%)
   vsc         :  3027 samples (  0.6%)
   vsc_ending  :   185 samples (  0.0%)
   yellow      : 21131 samples (  3.9%)
   Missing classes: [np.str_('unknown')]
✅ FixedVocabTrackStatusEncoder fitted
   Classes seen: ['green', 'red', 'safety_car', 'vsc', 'vsc_ending', 'yellow']
   Total classes: 7
   Output mode: integer labels


2025-07-13 12:17:48,142 - f1_etl - INFO - Imputing missing values before PCA
2025-07-13 12:17:48,203 - f1_etl - INFO - Using 6 components to capture 95.0% variance
2025-07-13 12:17:48,219 - f1_etl - INFO - PCA reduced features from 9 to 6
2025-07-13 12:17:48,219 - f1_etl - INFO - Explained variance ratio: [0.50715906 0.13257284 0.12536976 0.07228638 0.06798376 0.04801935]
2025-07-13 12:17:48,220 - f1_etl - INFO - Cumulative variance: [0.50715906 0.6397319  0.76510166 0.83738804 0.90537181 0.95339116]
2025-07-13 12:17:48,247 - f1_etl - INFO - Updated features for windowing: ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6']
2025-07-13 12:17:48,247 - f1_etl - INFO - Processing 538556 total telemetry rows
2025-07-13 12:17:48,248 - f1_etl - INFO - Grouping by: ['SessionId', 'Driver']
2025-07-13 12:17:50,633 - f1_etl - INFO - Total sequences generated: 10760
2025-07-13 12:17:50,664 - f1_etl - INFO - Generated 10760 sequences with shape (10760, 100, 6)
2025-07-13 12:17:50,671 - f1_etl - INFO - No mi

In [13]:
dataset['X'].shape

(10760, 100, 6)

In [14]:
splits = prepare_data_with_validation(dataset, val_size=0.00, test_size=0.30)
class_names = list(dataset["label_encoder"].class_to_idx.keys())

# 5. Train model

# model_name = f"rocket_driver{"_".join(drivers)}"
model_name = f"rocket_pca_drivers_all"
model = RocketClassifier(n_kernels=1000)


=== DATA SPLIT SUMMARY ===
Total samples: 10,760
Train: 7,531 (70.0%)
Val:   None (skipped)
Test:  3,129 (29.1%) - removed 100 samples

Train class distribution:
  Class 0: 6,364 (84.5%)
  Class 1: 289 (3.8%)
  Class 2: 575 (7.6%)
  Class 4: 48 (0.6%)
  Class 5: 1 (0.0%)
  Class 6: 254 (3.4%)

Test class distribution:
  Class 0: 2,578 (82.4%)
  Class 1: 217 (6.9%)
  Class 2: 155 (5.0%)
  Class 4: 14 (0.4%)
  Class 5: 2 (0.1%)
  Class 6: 163 (5.2%)


In [15]:
run_id = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{model_name}"

evaluator = ModelEvaluationSuite(
    output_dir="evaluation_results",
    run_id=run_id,
)

dataset_metadata = create_metadata_from_f1_dataset(
    data_config=data_config,
    dataset=dataset,
    features_used="pca",
)

model_metadata = create_model_metadata(
    model_name=model_name,
    model=model,
)

training_results = train_and_validate_model(
    model=model,
    splits=splits,
    class_names=class_names,
    evaluator=evaluator,
    dataset_metadata=dataset_metadata,
    model_metadata=model_metadata,
)


TRAINING WITH TEST: rocket_pca_drivers_all
Training on train set...

Validation set not available (val_size=0.0)

Running full evaluation on test set...

EVALUATING: ROCKET_PCA_DRIVERS_ALL
Evaluation ID: rocket_pca_drivers_all_20250713_121858_rocket_pca_drivers_all
Training model...
Generating predictions...

📊 OVERALL PERFORMANCE
Accuracy:    0.7725
F1-Macro:    0.2037
F1-Weighted: 0.7363

🎯 TARGET CLASS ANALYSIS: SAFETY_CAR
Precision:       0.3592
Recall:          0.3290
F1-Score:        0.3434
True Positives:    51
False Negatives:  104 (missed safety_car events)
False Positives:   91 (false safety_car alarms)
True Negatives:  2883

📈 PER-CLASS PERFORMANCE
green       : P=0.832, R=0.917, F1=0.873, N=2578
red         : P=0.009, R=0.005, F1=0.006, N=217
safety_car  : P=0.359, R=0.329, F1=0.343, N=155
vsc         : P=0.000, R=0.000, F1=0.000, N=14
vsc_ending  : P=0.000, R=0.000, F1=0.000, N=2
yellow      : P=0.000, R=0.000, F1=0.000, N=163

🔍 CONFUSION MATRIX
                 Pred_gre

In [17]:
# 6. Evaluate on external dataset
external_config = DataConfig(
    sessions=[
        SessionConfig(2024, "Canadian Grand Prix", "R"),
        SessionConfig(2024, "Saudi Arabian Grand Prix", "R"),
    ],
    drivers=drivers,  # Use same drivers as training
    include_weather=False,
)

# Using PCA with automatic component selection (95% variance)
external_dataset = create_safety_car_dataset(
    config=external_config,
    window_size=window_size,
    prediction_horizon=prediction_horizon,
    normalize=True,
    target_column="TrackStatus",
    feature_transform="pca",  # Enable PCA
    pca_components=6,
)

2025-07-13 12:20:51,880 - f1_etl - INFO - Preprocessing configuration:
2025-07-13 12:20:51,882 - f1_etl - INFO -   Missing values: enabled (forward_fill)
2025-07-13 12:20:51,882 - f1_etl - INFO -   Normalization: enabled (standard)
2025-07-13 12:20:51,882 - f1_etl - INFO -   Feature transform: pca
2025-07-13 12:20:51,883 - f1_etl - INFO -   Resampling: disabled
2025-07-13 12:20:51,883 - f1_etl - INFO - Driver configuration:
2025-07-13 12:20:51,883 - f1_etl - INFO -   Global drivers: ['1']
2025-07-13 12:20:51,884 - f1_etl - INFO -   Canadian Grand Prix: ['1']
2025-07-13 12:20:51,884 - f1_etl - INFO -   Saudi Arabian Grand Prix: ['1']
core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Loading session: 2024 Canadian Grand Prix R


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '63', '44', '81', '14', '18', '3', '10', '31', '27', '20', '77', '22', '24', '55', '23', '11', '16', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Loading session: 2024 Saudi Arabian Grand Prix R


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
2025-07-13 12:20:57,904 - f1_etl - INFO - Creating new fixed vocabulary encoder
2025-07-13 12:20:57,936 - f1_etl - INFO - Applying PCA transformation to telemetry features
2025-07-13 12:20:57,937 - f1_etl - INFO - Applying PCA to 9 numeric featur


📊 Track Status Analysis (training_data):
   green       : 101246 samples ( 86.3%)
   safety_car  : 11292 samples (  9.6%)
   yellow      :  4721 samples (  4.0%)
   Missing classes: [np.str_('red'), np.str_('unknown'), np.str_('vsc'), np.str_('vsc_ending')]
✅ FixedVocabTrackStatusEncoder fitted
   Classes seen: ['green', 'safety_car', 'yellow']
   Total classes: 7
   Output mode: integer labels


2025-07-13 12:20:58,554 - f1_etl - INFO - Total sequences generated: 2342
2025-07-13 12:20:58,559 - f1_etl - INFO - Generated 2342 sequences with shape (2342, 100, 6)
2025-07-13 12:20:58,561 - f1_etl - INFO - No missing values detected, skipping imputation
2025-07-13 12:20:58,561 - f1_etl - INFO - Skipping normalization for PCA features (already standardized)
2025-07-13 12:20:58,564 - f1_etl - INFO - Final dataset summary:
2025-07-13 12:20:58,564 - f1_etl - INFO -   Sequences: 2342
2025-07-13 12:20:58,564 - f1_etl - INFO -   Features: 6
2025-07-13 12:20:58,565 - f1_etl - INFO -   Classes: 7 (integer)
2025-07-13 12:20:58,565 - f1_etl - INFO -   Label shape: (2342,)
2025-07-13 12:20:58,565 - f1_etl - INFO -     green       :  2021 samples ( 86.3%)
2025-07-13 12:20:58,565 - f1_etl - INFO -     safety_car  :   225 samples (  9.6%)
2025-07-13 12:20:58,566 - f1_etl - INFO -     yellow      :    96 samples (  4.1%)


In [18]:
external_results = evaluate_on_external_dataset(
    trained_model=training_results["model"],
    external_config=external_config,
    original_dataset_metadata=dataset_metadata,
    model_metadata=model_metadata,
    class_names=class_names,
    evaluator=evaluator,
    resampling_strategy=dataset_metadata.resampling_strategy,
    resampling_config=dataset_metadata.resampling_config,
    original_dataset_config=dataset['config']
)

2025-07-13 12:21:03,830 - f1_etl - INFO - Preprocessing configuration:
2025-07-13 12:21:03,834 - f1_etl - INFO -   Missing values: enabled (forward_fill)
2025-07-13 12:21:03,835 - f1_etl - INFO -   Normalization: enabled (standard)
2025-07-13 12:21:03,835 - f1_etl - INFO -   Feature transform: pca
2025-07-13 12:21:03,836 - f1_etl - INFO -   Resampling: disabled
2025-07-13 12:21:03,836 - f1_etl - INFO - Driver configuration:
2025-07-13 12:21:03,836 - f1_etl - INFO -   Global drivers: ['1']
2025-07-13 12:21:03,837 - f1_etl - INFO -   Canadian Grand Prix: ['1']
2025-07-13 12:21:03,837 - f1_etl - INFO -   Saudi Arabian Grand Prix: ['1']
core           INFO 	Loading data for Canadian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_da


EXTERNAL DATASET EVALUATION
Using preprocessing from training dataset:
  Feature transform: pca
  PCA components: 6
Loading external dataset...
Loading session: 2024 Canadian Grand Prix R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '63', '44', '81', '14', '18', '3', '10', '31', '27', '20', '77', '22', '24', '55', '23', '11', '16', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


Loading session: 2024 Saudi Arabian Grand Prix R


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '81', '14', '63', '38', '4', '44', '27', '23', '20', '31', '2', '22', '3', '77', '24', '18', '10']
2025-07-13 12:21:07,268 - f1_etl - INFO - Creating new fixed vocabulary encoder
2025-07-13 12:21:07,302 - f1_etl - INFO - Applying PCA transformation to telemetry features
2025-07-13 12:21:07,303 - f1_etl - INFO - Applying PCA to 9 numeric features: ['Speed', 'RPM', 'nGear', 'Throttle', 'Brake', 'X', 'Y', 'Distance', 'DifferentialDistance']
2025-07-13 12:21:07,344 - f1_etl - INFO - Using 6 components (user specified)
2025-07-13 12:21:07,350 - f1_etl - INFO - PCA reduced features from 9 to 6
2025-07-13 12:21:07,350 - f1_etl - INFO - Explained variance ratio: [0.49247878 0.14273236 0.13130


📊 Track Status Analysis (training_data):
   green       : 101246 samples ( 86.3%)
   safety_car  : 11292 samples (  9.6%)
   yellow      :  4721 samples (  4.0%)
   Missing classes: [np.str_('red'), np.str_('unknown'), np.str_('vsc'), np.str_('vsc_ending')]
✅ FixedVocabTrackStatusEncoder fitted
   Classes seen: ['green', 'safety_car', 'yellow']
   Total classes: 7
   Output mode: integer labels


2025-07-13 12:21:07,998 - f1_etl - INFO - Total sequences generated: 2342
2025-07-13 12:21:08,003 - f1_etl - INFO - Generated 2342 sequences with shape (2342, 100, 6)
2025-07-13 12:21:08,004 - f1_etl - INFO - No missing values detected, skipping imputation
2025-07-13 12:21:08,005 - f1_etl - INFO - Skipping normalization for PCA features (already standardized)
2025-07-13 12:21:08,008 - f1_etl - INFO - Final dataset summary:
2025-07-13 12:21:08,008 - f1_etl - INFO -   Sequences: 2342
2025-07-13 12:21:08,008 - f1_etl - INFO -   Features: 6
2025-07-13 12:21:08,009 - f1_etl - INFO -   Classes: 7 (integer)
2025-07-13 12:21:08,009 - f1_etl - INFO -   Label shape: (2342,)
2025-07-13 12:21:08,009 - f1_etl - INFO -     green       :  2021 samples ( 86.3%)
2025-07-13 12:21:08,010 - f1_etl - INFO -     safety_car  :   225 samples (  9.6%)
2025-07-13 12:21:08,010 - f1_etl - INFO -     yellow      :    96 samples (  4.1%)


External dataset size: 2,342 samples
Generating predictions...

📊 OVERALL PERFORMANCE
Accuracy:    0.7071
F1-Macro:    0.3249
F1-Weighted: 0.7530

🎯 TARGET CLASS ANALYSIS: SAFETY_CAR
Precision:       0.3785
Recall:          0.6578
F1-Score:        0.4805
True Positives:   148
False Negatives:   77 (missed safety_car events)
False Positives:  243 (false safety_car alarms)
True Negatives:  1874

📈 PER-CLASS PERFORMANCE
green       : P=0.908, R=0.746, F1=0.819, N=2021
red         : P=0.000, R=0.000, F1=0.000, N=0
safety_car  : P=0.379, R=0.658, F1=0.481, N=225
yellow      : P=0.000, R=0.000, F1=0.000, N=96

🔍 CONFUSION MATRIX
                 Pred_green  Pred_red  Pred_safety_car  Pred_yellow
True_green             1508        92              234          187
True_red                  0         0                0            0
True_safety_car          68         9              148            0
True_yellow              85         2                9            0

💾 Results saved:
  Complete: